Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error
import pickle
import sys

C:\Users\mokky\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\mokky\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\mokky\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\mokky\AppData\Roaming\Python\Python37\site-

In [2]:
import tensorflow as tf
tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [3]:
# Function to show various error measurement
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    mse = np.mean((forecast - actual)**2)  # MSE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    #corr = np.corrcoef(forecast, actual)[0,1]   # corr
    #mins = np.amin(np.hstack([forecast[:,None], 
    #                          actual[:,None]]), axis=1)
    #maxs = np.amax(np.hstack([forecast[:,None], 
    #                          actual[:,None]]), axis=1)
    #minmax = 1 - np.mean(mins/maxs)             # minmax
    #acf1 = acf(fc-test)[1]                      # ACF1
    #return({'mape':mape, 'me':me, 'mae': mae, 
    #        'mpe': mpe, 'rmse':rmse, 'acf1':acf1, 
    #        'corr':corr, 'minmax':minmax})
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'mse':mse, 'rmse':rmse})

In [4]:
import os
os.chdir('d:/Users/mokky/Documents/GitHub/nus-iss/PRS-PM-ISY5002-GROUP5/SystemCode')
print(os.getcwd())

d:\Users\mokky\Documents\GitHub\nus-iss\PRS-PM-ISY5002-GROUP5\SystemCode


In [5]:
file_dir = './Data Exploration/data/uc3/'
X_train_file_path = 'all_hosp_new_train_X_uc3.pkl'
y_train_file_path = 'all_hosp_data_uc3_y_train_uc3.pkl'
X_test_file_path = 'all_hosp_new_test_X_uc3.pkl'
y_test_file_path = 'all_hosp_data_uc3_y_test_uc3.pkl'

In [6]:
df_X_train = pd.read_pickle(file_dir + X_train_file_path)
df_y_train = pd.read_pickle(file_dir + y_train_file_path)
df_X_test = pd.read_pickle(file_dir + X_test_file_path)
df_y_test = pd.read_pickle(file_dir + y_test_file_path)
print(df_X_train.shape, df_y_train.shape, df_X_test.shape, df_y_test.shape)

(46888, 14313) (46888,) (15629, 14313) (15629,)


In [7]:
print(df_y_test.values.shape)

(15629,)


Neural Networks ( Sequential )

In [8]:
NN_model = Sequential()
n_hid_nodes = 4096
dropout_rate = 0.2
rand_seed = 42
kernel_init = 'he_normal'
hid_act = 'relu'

In [9]:
NN_model.add(Dense(n_hid_nodes, kernel_initializer=kernel_init,input_dim = df_X_train.shape[1], activation=hid_act))
NN_model.add(Dropout(dropout_rate, seed=rand_seed))

In [10]:
#NN_model.add(GaussianNoise(0.01,input_shape = (df_X_train.shape[1],)))
#NN_model.add(Dense(n_hid_nodes, kernel_initializer=kernel_init,activation='relu'))
#NN_model.add(Dropout(dropout_rate, seed=rand_seed))

In [11]:
for i in range(7):
    NN_model.add(Dense(n_hid_nodes, kernel_initializer=kernel_init))#,activation=hid_act))
    NN_model.add(BatchNormalization())
    NN_model.add(Activation(hid_act))
    NN_model.add(Dropout(dropout_rate, seed=rand_seed))

In [12]:
NN_model.add(Dense(1, kernel_initializer=kernel_init,activation='linear'))

In [13]:
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4096)              58630144  
_________________________________________________________________
dropout (Dropout)            (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
batch_normalization (BatchNo (None, 4096)              16384     
_________________________________________________________________
activation (Activation)      (None, 4096)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              1

In [14]:
checkpoint_name = 'NN2_Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [ ]:
NN_model.fit(df_X_train, df_y_train, epochs=100, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

# save model to file
NN_model.save(file_dir + "uc3_NN2_model.h5")

Train on 37510 samples, validate on 9378 samples
Epoch 1/100
37504/37510 [============================>.] - ETA: 0s - loss: 4258.3989 - mean_absolute_error: 4258.3970
Epoch 00001: val_loss improved from inf to 2315.75026, saving model to NN2_Weights-001--2315.75026.hdf5
37510/37510 [==============================] - 159s 4ms/sample - loss: 4257.9636 - mean_absolute_error: 4257.9619 - val_loss: 2315.7503 - val_mean_absolute_error: 2315.7510
Epoch 2/100
37504/37510 [============================>.] - ETA: 0s - loss: 2558.1660 - mean_absolute_error: 2558.1665
Epoch 00002: val_loss improved from 2315.75026 to 2039.10448, saving model to NN2_Weights-002--2039.10448.hdf5
37510/37510 [==============================] - 158s 4ms/sample - loss: 2558.1961 - mean_absolute_error: 2558.1968 - val_loss: 2039.1045 - val_mean_absolute_error: 2039.1045
Epoch 3/100
37504/37510 [============================>.] - ETA: 0s - loss: 2373.9062 - mean_absolute_error: 2373.9077
Epoch 00003: val_loss improved from 

In [ ]:
predictions = NN_model.predict(df_X_test)

In [ ]:
sns.regplot(x=df_y_test, y=predictions)

In [ ]:
# MSE
print('Mean squared error: %.2f'
      % mean_squared_error(df_y_test,predictions, squared=True))

# RMSE
print('Root Mean squared error: %.2f'
      % mean_squared_error(df_y_test,predictions, squared=False))

# MAE
print('Mean absolute error: %.2f'
      % mean_absolute_error(df_y_test,predictions))

In [ ]:
forecast_accuracy(predictions.reshape((-1,)), df_y_test)